In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler 
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import pickle

##

C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Load and Divide Data into Training and Testing  
  
In addition, the data is scaled. 


In [2]:
# Load comma delimited sensor data into Pandas data frame
sensor_file = "../data/sensor_11_12_2017_10_52_24.csv"
df = pd.read_csv(sensor_file)
print (df.shape)

# Extract features and target (y)
X = df.iloc[:,1:15] # exclude data
y = df.iloc[:,16]
print (X.shape)
print (y.shape)

# Use 80% of data for training, 20% for testing
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20, random_state = 1)
print (X_train.shape)
print (X_test.shape)

# Scaling features, mean = 0, stddev = 1
scaler = StandardScaler()
scaler.fit(X_train)  
X_train = scaler.transform(X_train)

# Head
df.head()

##

(5457, 17)
(5457, 14)
(5457L,)
(4365, 14)
(1092, 14)


,DateTime,Temp,Humidity,Pressure,Yaw,Pitch,Roll,MagX,MagY,MagZ,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,State
0,11/12/2017 10:52:35.251176,29.361,40.702,1031.956,215.801975,356.730118,2.171165,-5.351357,5.945352,26.308798,0.056247,0.034719,0.960166,-0.060679,0.072064,0.049957,standing
1,11/12/2017 10:52:35.523388,29.308,39.749,1031.952,216.099403,356.626506,2.103062,-10.720130,12.090914,53.852757,0.055035,0.016267,0.974544,0.011399,0.019521,-0.016927,standing
2,11/12/2017 10:52:35.688238,29.468,40.231,1031.941,216.340904,356.301112,2.310486,-13.004756,14.781075,65.216164,0.046307,0.014810,0.939451,0.040319,0.005237,-0.007642,standing
3,11/12/2017 10:52:35.849093,29.397,40.121,1031.950,216.613607,356.519507,2.298305,-13.880388,16.216396,69.694740,0.060854,0.016267,0.945544,-0.005668,0.020673,0.007911,standing
4,11/12/2017 10:52:36.010403,29.343,40.088,1031.943,216.940253,356.745464,2.196436,-13.895806,16.390114,71.753525,0.055763,0.023308,0.937014,-0.008612,0.015847,0.017006,standing


In [110]:
### Practice of conversions



# prediction = loaded_model.predict(array(dat).reshape(1,-1))


# print "Type", type(X_test)
# print "Shape", X_test.shape
# print X_test.head(1)

# pd_series = X_test.iloc[10,:]
# print "Type", type(pd_series)
# print "Shape", pd_series.shape
# print pd_series #.head(3)

# np = pd_series.reshape(1,-1)
# print "Type", type(np)
# print "Shape", np.shape
# print np

# print pd_series.reshape(1,-1).shape
# print
# print X_test.iloc[10,:].shape


# mylist = list(pd_series)
# print "Type", type(mylist)
# print "Len", len(mylist)

# Let's format floats to 3 digits
# fx = lambda x: ('%.3f' % float(x))
# mylist_formatted = map(fx, mylist)
# print mylist_formatted

# from numpy import array
# mylist_array = array(mylist)
# print "Type", type(mylist_array)
# print "Shape", mylist_array.shape
# print map(fx,mylist_array)

dat = ['15.000', '20.323', '511.239', '82.007', '179.526', '0.073', '-12.106', '-3.473', '31.989', '0.007', '0.001', '0.473', '-0.000', '0.000', '0.000']
dat_series = pd.Series(dat)
print "Type", type(dat_series)
print "Shape", dat_series.shape
print "Type", type(dat_series.reshape(1,-1))
# print dat_series.reshape(1,15)
print dat_series.reshape(1,-1).shape

Type <class 'pandas.core.series.Series'>
Shape (15L,)
Type <type 'numpy.ndarray'>
(1L, 15L)


C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:46: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:48: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


# Construct Neural Network Model and Tweak

In [54]:
# Supervised neural network model
# http://scikit-learn.org/stable/modules/neural_networks_supervised.html

# NN has 3 hidden layers. 1st layer has 15 neurons which equals 15 features
#     2nd layer has 100 (just because), and 3rd layer has 8 which equals 8 classes

# Pipeline, push scaler and classifer
pipe = Pipeline([('scl', StandardScaler()),
                 ('clf', MLPClassifier(solver='lbfgs', alpha=0.00001,
                                           hidden_layer_sizes=(15,100, 8), random_state=1))])

# How well does the model converge? Overfitting? Underfitting?
scores = cross_val_score(estimator=pipe, X=X_train, y=y_train,cv=10,n_jobs=1)
for s in scores:
    print (s)
    
# The final CV acc for training data was 0.64 using Logistic Regression
print ('CV Acc: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

##

0.904545454545
0.906605922551
0.908675799087
0.915525114155
0.920091324201
0.92183908046
0.905747126437
0.935483870968
0.923963133641
0.907834101382


TypeError: only integer scalar arrays can be converted to a scalar index

# Use Models to Make Predictions

In [55]:
# Fit training data
pipe.fit(X_train, y_train)
predictions = pipe.predict(scaler.transform(X_test)) # Get predictions

##

# Evaluate Model Performance on Testing Data

In [56]:
# Better table
pd.crosstab(y_test, predictions)

# Ideally, there would zeros everywhere except for the main diagonal
# Last row, for example, the actual state is 'walking', but the model
# is predicting 'turn-right'

##

col_0,go_down,go_up,spin_left,spin_right,standing,turn_left,turn_right,walking
State,,,,,,,,
go_down,29,9,1,0,5,0,5,0
go_up,4,44,2,1,0,0,1,1
spin_left,0,1,100,0,0,3,0,0
spin_right,0,0,0,66,1,0,0,0
standing,0,2,0,0,125,0,0,3
turn_left,1,1,0,0,5,155,6,9
turn_right,0,0,0,1,2,2,138,14
walking,0,1,0,0,4,4,9,337


In [57]:
# Metrics
precision, recall, fscore, support = score(y_test, predictions)

print ('Prec ','Rec','  Fscore')
for i in range(len(precision)):
    print (('%.3f' % precision[i]), ('%.3f' % recall[i]), ('%.3f' % fscore[i]))
    
## Note that accuracy is not appropriate for unbalanced classes.
## F-score is the superior metric
## The lowest Fscores are associated with 'go_down' and 'go_up'

##

('Prec ', 'Rec', '  Fscore')
('0.853', '0.592', '0.699')
('0.759', '0.830', '0.793')
('0.971', '0.962', '0.966')
('0.971', '0.985', '0.978')
('0.880', '0.962', '0.919')
('0.945', '0.876', '0.909')
('0.868', '0.879', '0.873')
('0.926', '0.949', '0.937')


# Deploying Demonstration  
  
The following code demonstrate how the Neural Network model is saved and deployed to automatically classify sensor readings into one of eight classes.  

I intend on loading this model to the Raspberry Pi and feeding it with the sensor data to indicate automatically what motion the user is making.


In [58]:
# Use pickle library to save model
filename = '../model/sensor_model_2017.11.20a.sav'
pickle.dump(pipe, open(filename, 'wb'))


In [99]:
# NOTE: For reference only
#Add this to data logger for automatically describing the state of motion
filename = '../model/sensor_model_2017.11.20a.sav'

# Load our saved model
loaded_model = pickle.load(open(filename, 'rb'))
# Scaling features, mean = 0, stddev = 1
scaler = StandardScaler()
scaler.fit(X_test)
# Simulate 10 consecutive sensor readings and make a prediction
dat = ['15.000', '20.323', '511.239', '82.007', '179.526', '0.073', '-12.106', '-3.473', '31.989', '0.007', '0.001', '0.473', '-0.000', '0.000', '0.000']
print len(dat)
# prediction = loaded_model.predict(pd.Series(dat))
# print (prediction)


# prediction = loaded_model.predict(scaler.transform(X_test.iloc[0:5,:].reshape(1,-1)))
# print (prediction)
print

for i in range(100,110):
    prediction = loaded_model.predict(scaler.transform(X_test.iloc[i,:].reshape(1,-1))) 

    print (prediction), type(X_test.iloc[i,:]), type(X_test.iloc[i,:].reshape(1,-1))
    
    
## It may be necessary to calculate a running average on sensors to minimize
## noise.

##

15

['walking'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['walking'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['turn_left'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['spin_left'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['turn_right'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['spin_right'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['spin_right'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['go_up'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['standing'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>
['walking'] <class 'pandas.core.series.Series'> <type 'numpy.ndarray'>


C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [72]:
print list(X_test.columns)

[' Temp', ' Humidity', ' Pressure', ' Yaw', ' Pitch', ' Roll', ' MagX', ' MagY', ' MagZ', ' AccX', ' AccY', ' AccZ', ' GyroX', ' GyroY']


In [114]:
mydata = [['11.406', '22.902', '514.395', '81.636', '179.822', '179.898', '-14.837', '-4.425', '34.947', '0.004', '-0.003', '0.471', '-0.000', '-0.001', '-0.001'],
['11.409', '22.982', '514.394', '81.668', '179.818', '179.895', '-14.865', '-4.491', '34.961', '0.003', '-0.003', '0.472', '-0.000', '-0.000', '-0.001'],
['11.418', '22.867', '514.394', '81.683', '179.817', '179.892', '-14.835', '-4.611', '34.928', '0.003', '-0.003', '0.472', '-0.000', '-0.001', '-0.001']]

filename = '../model/sensor_model_2017.11.20a.sav'

# Load our saved model
loaded_model = pickle.load(open(filename, 'rb'))
# Scaling features, mean = 0, stddev = 1
scaler = StandardScaler()

header = ['Temp','Humidity','Pressure','Yaw','Pitch','Roll','MagX','MagY','MagZ','AccX','AccY','AccZ','GyroX',' GyroY','GyroZ']
print len(header), len(mydata[0])
print mydata[0]
print header

# scaler.fit(X_test) # Fit with data frame
my_df = pd.DataFrame(mydata, columns=header)
my_df.head(2)
scaler.fit(my_df)

# Simulate 10 consecutive sensor readings and make a prediction
dat = ['15.000', '20.323', '511.239', '82.007', '179.526', '0.073', '-12.106', '-3.473', '31.989', '0.007', '0.001', '0.473', '-0.000', '0.000', '0.000']
prediction = loaded_model.predict(pd.Series(dat).reshape(1,-1))
# print (prediction)

# dat_series = pd.Series(dat)
# print "Type", type(dat_series)
# print "Shape", dat_series.shape
# print "Type", type(dat_series.reshape(1,-1))
# # print dat_series.reshape(1,15)
# print dat_series.reshape(1,-1).shape

# prediction = loaded_model.predict(scaler.transform(X_test.iloc[5,:].reshape(1,-1)))
# print (prediction)
    

15 15
['11.406', '22.902', '514.395', '81.636', '179.822', '179.898', '-14.837', '-4.425', '34.947', '0.004', '-0.003', '0.471', '-0.000', '-0.001', '-0.001']
['Temp', 'Humidity', 'Pressure', 'Yaw', 'Pitch', 'Roll', 'MagX', 'MagY', 'MagZ', 'AccX', 'AccY', 'AccZ', 'GyroX', ' GyroY', 'GyroZ']


C:\Users\Chuck B\Anaconda3\envs\py27\lib\site-packages\ipykernel\__main__.py:24: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


ValueError: operands could not be broadcast together with shapes (1,15) (14,) (1,15) 

# Compares Before and After Reshaping

In [43]:
# Reference only. In the next cell the data must be reshaped. Here are the differences.
print (X_test.iloc[1,:])
print (X_test.iloc[1,:].reshape(1,-1))

##

 Temp          25.527000
 Humidity      36.760000
 Pressure    1025.950000
 Yaw          158.921194
 Pitch        356.681997
 Roll           2.092825
 MagX         -42.919319
 MagY         -14.954748
 MagZ          76.645782
 AccX           0.042428
 AccY           0.022094
 AccZ           0.943594
 GyroX          0.000503
 GyroY          0.017684
Name: 3196, dtype: float64
[[  2.55270000e+01   3.67600000e+01   1.02595000e+03   1.58921194e+02
    3.56681997e+02   2.09282500e+00  -4.29193190e+01  -1.49547480e+01
    7.66457820e+01   4.24280000e-02   2.20940000e-02   9.43594000e-01
    5.03000000e-04   1.76840000e-02]]


C:\Users\Chuck B\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
